<b style="font-size:2em;">Final Project Submission</b>

* Student name: Edward Amor 
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: Victor Geislinger
* Blog post URL: 

# Business Understanding

## Background

A real estate company in the King County area wants to investigate house sales in order to predict pricing. Our task is to clean, explore, and model the housing data they have supplied to us in `kc_house_data.csv`, and identify house features which significantly effect house price. 

## Scope

To identify which house features significantly effect house price, we will use standard EDA practices, hypothesis testing, and regression modeling to come to our conclusions.

For our client's consumption the results of our analysis will be condensed into a presentation. 

## Goal

Our goal is to generate a multivariate linear regression model which as accurately as possible predicts the sale price of houses.  

## Objectives

1. Data Acquisition
2. Data Understanding
  - Data Cleaning
  - Data Exploration
  - Data Visualization
3. Modeling
  - Feature Engineering
    - Feature Transformation
    - Feature Selection
  - Training
  - Model Evaluation

# Data Acquisition

The data for this project can be found in the file `kc_house_data.csv`.

Within this repository is also `data_dictionary.csv` which describes the data features.

# Data Understanding

According to our data dictionary the features of our dataset are as follows:

| column        | description                                                  |
| ------------- | ------------------------------------------------------------ |
| id            | unique identified for a house                                |
| date          | Date house was sold                                          |
| price         | Price is prediction target                                   |
| bedrooms      | Number of Bedrooms/House                                     |
| bathrooms     | Number of bathrooms/bedrooms                                 |
| sqft_living   | square footage of the home                                   |
| sqft_lot      | square footage of the lot                                    |
| floors        | Total floors (levels) in house                               |
| waterfront    | House which has a view to a waterfront                       |
| view          | Has been viewed                                              |
| condition     | How good the condition is ( Overall )                        |
| grade         | overall grade given to the housing unit                      |
| sqft_above    | square footage of house apart from basement                  |
| sqft_basement | square footage of the basement                               |
| yr_built      | Built Year                                                   |
| yr_renovated  | Year when house was renovated                                |
| zipcode       | zip                                                          |
| lat           | Latitude coordinate                                          |
| long          | Longitude coordinate                                         |
| sqft_living15 | The square footage of interior housing living space for the nearest 15 neighbors |
| sqft_lot15    | The square footage of the land lots of the nearest 15 neighbors |

Of the 20 columns most notable is the price feature which will be our target during linear regression.

Another thing to note is we have location data. This could be used to provide some insight into pricing by location, but may also be of use when delivering key information to our client.

## Data Cleaning

For this section we will inspect our dataset cleaning up inconsistencies such as, null values, duplicates, and incorrect data types.

In [ ]:
# import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

In [ ]:
# read in dataframe and output dataframe info
df = pd.read_csv("kc_house_data.csv")
df.info()

Our raw dataset contains **21597 records**, and has **21 features**.

The data types found within our dataset are: `float64`, `int64`, `object`.

Next we will inspect our features more closely, especially the ones which are of `object` data type. Just to make sure they're the correct data type.

In [ ]:
# Output the first 5 rows
display(df.head(5))

The `date` column should be transformed to the `datetime` data type instead of `object`.

The `sqft_basement` column needs to be further inspected, the values appear to be `float` there may be some `string` values.

In [ ]:
# convert date column from string data type to a datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# inspect the values of the sqft_basement column
display(df['sqft_basement'].value_counts())

There is a '?' value, this may mean that the value is unknown. We will replace this with a null value, and then convert the column into a float.

In [ ]:
# replace ? with np.nan in the sqft_basement column and convert the column to float
df['sqft_basement'] = df['sqft_basement'].replace('?', np.nan).astype('float64')

In [ ]:
# redisplay the values in the sqft_basement column
display(df['sqft_basement'].value_counts())

All our columns are now numeric, it's time to inspect for any interesting anomalies.

### Inspecting Distributions

In [ ]:
# plot a distribution plot for each feature
fig, ax = plt.subplots(7, 3, figsize=(16, 9*3)) # 21 sub plots

for col, axes in zip(df.columns, ax.flatten()):
    sns.distplot(df[col], kde=False, rug=True, ax=axes)

plt.show()

Columns to disregard when cleaning:

- `id`
- `date`
- `long`
- `lat`
- `zipcode`

From the histograms it looks like we may want to drop the `yr_renovated` column, we will have to investigate further.

There are quite a few columns which should be transformed into the `categorical` data type.

### Impute Inconsistencies

#### yr_renovated

In [ ]:
# inspect the yr_renovated column by plotting a violinplot
fig, ax = plt.subplots(figsize=(16, 9))

sns.violinplot('yr_renovated', data=df, width=.5)
plt.show()

In [ ]:
# check the value counts for the y_renovated column
display(df['yr_renovated'].value_counts().head())

An extremely large percentage of the `yr_renovated` column has the value 0, meaning no renovation has occurred. Instead of removing the column, we can convert the values other than 0 to a 1. Giving us a usable feature instead of dropping one.

In [ ]:
# transform the values other than 0 to a 1 in yr_renovated, and rename the column
mask = df['yr_renovated'] != 0
df.loc[mask, 'yr_renovated'] = 1
df.rename(columns={'yr_renovated': 'renovated'}, inplace=True)

#### sqft_basement

In [ ]:
# inspect the sqft_basement column by plotting a violinplot
fig, ax = plt.subplots(figsize=(16, 9))

sns.violinplot('sqft_basement', data=df, width=.5)
plt.show()

In [ ]:
# check the value counts for the sqft_basement column
display(df['sqft_basement'].value_counts().head())

An extremely large percentage of the `yr_renovated` column has the value 0, meaning no renovation has occurred. Instead of removing the column, we can convert the values other than 0 to a 1. Giving us a usable feature instead of dropping one.

In [ ]:
# transform the values other than 0 to a 1 in yr_renovated, and rename the column
mask = df['sqft_basement'] != 0
df.loc[mask, 'sqft_basement'] = 1
df.rename(columns={'sqft_basement': 'basement'}, inplace=True)

#### bedrooms

Another column we see an error in is the `bedrooms` columns. There is a huge skew, maybe due to a typing error.

In [ ]:
# output value counts
display(df['bedrooms'].value_counts())

The house with 33 bedrooms may be an typing error, most likely a room with 3 bedrooms, not 33 which would be highly unusual.

In [ ]:
# change the 33 bedroom house to a 3 bedroom
df.loc[df['bedrooms'] == 33, 'bedrooms'] = 3

### NA Values

In [ ]:
# Output how many NA values we have
display(df.isna().sum())

For the `waterfront` and `view` columns, we will replace the na values with 0, to signify no waterfront and no views respectively.

For the `sqft_basement` column we will replace the na values with 0, to signify there is no basement.

In [ ]:
# replace the na values with 0 in waterfront, view, sqft_basement
df.replace(np.nan, 0, inplace=True)
display(df.isna().sum())

### Outliers

Since we've taken care of our null values, now we can focus on taking care of outliers which will significantly affect our linear regression in the future.

We will first start with our `price` feature.

In [ ]:
# show boxplot of price column
fig, ax = plt.subplots(figsize=(16, 5))

sns.boxplot('price', data=df, ax=ax, showmeans=True)

plt.show()

There appears to be a significant amount of outliers. Let's identify how many, and whether we should eliminate them.

In [ ]:
# using iqr find outliers
q1, q3 = np.quantile(df['price'], (.25, .75))
iqr = q3 - q1
low_bd, up_bd = q1 - iqr*1.5, q3 + iqr*1.5 # upper and lower bound of range
mask = (df['price'] < low_bd) | (df['price'] > up_bd) # outliers mask

outliers_mask = df.index[mask].to_numpy()

In [ ]:
# count our outliers
display(f"Outliers Count: {mask.sum()}")
display(f"Outliers Percentage: {mask.sum()/df.shape[0]:.0%}")

Since we would only be removing approximately 5% of our dataset, 1158 data points, we will drop those rows.

In [ ]:
# drop the data points which are price outliers and display a new boxplot of price
df.drop(index=outliers_mask, inplace=True, errors='ignore') # ignore to allow re run

fig, ax = plt.subplots(figsize=(16, 5))

sns.boxplot(df['price'])
plt.show()

We've successfully removed the large amount of `price` outliers that were previously in our dataset.

### Type Conversion

We will convert columns which should be categorical to their correct type.

- bedrooms
- bathrooms
- floors
- waterfront
- view
- condition
- grade
- renovated
- zipcode

In [ ]:
# access and convert the categorical columns
cat_cols = ['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view',
           'condition', 'grade', 'renovated', 'zipcode', 'yr_built']

df.loc[:, cat_cols] = df.loc[:, cat_cols].astype('category')

In [ ]:
# verify conversion
display(df.dtypes)

## Data Exploration

We'll use some guided questions to help us explore our data easier.

1. Using location data, where are the highest valued properties?
2. Are waterfront properties significantly expensive than non waterfront properties?
3. Is there a relationship between condition and pricing?

### Location

In [ ]:
# import our libraries
import plotly.express as px
from plotly.offline import init_notebook_mode

# connect to online cdn
init_notebook_mode(connected=True)

In [ ]:
# plot pricing of houses across King County
fig = px.scatter_mapbox(df.sort_values('price'), lat='lat', lon='long', color='price',
                       color_continuous_scale=px.colors.sequential.BuGn,
                       mapbox_style='carto-positron',
                       center=dict(lat=df.lat.median(), lon=df.long.median()),
                       title="House Prices in King County")
fig.show()

There appears to be higher concentrations of high valued houses towards the north of King County, With the majority of those located on the waterfront, there may be other contributing factors since there are high valued houses scattered around King County.

Let's inspect by zip code the median house price, to get a better picture at which areas have higher valued houses.

In [ ]:
# import json
import json

In [ ]:
# load in geojson of king county zip codes
with open('kc_zips.geojson') as f:
    geojson = json.load(f)

The King County zip codes geojson data was compiled from a larger dataset containing all US zip codes. The larger dataset can be found at: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html

The downloaded data was then converted to a 1.4 GB geojson file by using https://mapshaper.org/. After which it was filtered for only zip codes in King County. 

In [ ]:
# create grouped dataframe with median house price for each zip code
zipcode_df = df.groupby('zipcode').median().reset_index()
zipcode_df['zipcode'] = zipcode_df['zipcode'].astype(str)

In [ ]:
# plot median house price data on map by zipcode
fig = px.choropleth_mapbox(zipcode_df, geojson=geojson, 
                          featureidkey='properties.ZCTA5CE10', locations='zipcode', 
                          color='price', mapbox_style='carto-positron', opacity=.4,
                          center=dict(
                              lat=zipcode_df.lat.median(),
                              lon=zipcode_df.long.median()
                          ), color_continuous_scale=px.colors.sequential.BuGn,  
                          title="King County Median House Price by Zip Code", )
fig.show()

The map shows us that the 3 zip codes with the highest median house prices are located in the centrally. **Those 3 zip codes are 98039, 98040, and 98004**.

In the future it would be nice to reverse geocode the data we have, and see pricing by neighborhood.

### Waterfront Properties

In [ ]:
# immport our libraries
from scipy import stats

In [ ]:
# plot two distplots of the different waterfront price values
mask = df['waterfront'] == 1

fig, ax = plt.subplots(figsize=(16, 9))

sns.distplot(df.loc[mask, 'price'], norm_hist=True, ax=ax, label='waterfront')
sns.distplot(df.loc[~mask, 'price'], norm_hist=True, ax=ax, label='non-waterfront')

plt.show()

These two groups don't look too different, but we can use some hypothesis testing to verify our assumptions.

$\mu_0:$ Waterfront House Mean Value

$\mu_1:$ Non-Waterfront House Mean Value

$H_0: \mu_0 = \mu_1$

$H_a: \mu_0 \ge \mu_1$

$\alpha = $ 5%

In [ ]:
# seperate our two populations and also show some 5-point summary of them
mask = df['waterfront'] == 1
waterfront = df.loc[mask, 'price']
nonwaterfront = df.loc[~mask, 'price']

display(df.groupby('waterfront').describe()['price'])

Our means are different by about $200,000, but our standard deviations are approximately the same.

In [ ]:
# Run a welch's 2 sample t-test
results = stats.ttest_ind(waterfront, nonwaterfront, equal_var=False)
print(f"T-Stat: {results.statistic:.3f}", f"P-Value: {results.pvalue:.8%}", sep='\n')

It appears that we do have a statistically significantly difference between the two populations. For our stakeholders we will provide a simple visualization to drive the point home that waterfront properties are valued higher. 

In [ ]:
# plot barplot of the two populations average value wiht a confidence level of 95%
fig, ax = plt.subplots(figsize=(16, 9))

sns.barplot(x='waterfront', y='price', data=df, ax=ax)

plt.title('King County Average House Value of Waterfront/Non-Waterfront Properties')

plt.xticks([0, 1], ['Non-Waterfront', 'Waterfront'])
locs, labels = plt.yticks()
plt.yticks(locs, [f"{loc/1000:n}" for loc in locs])

plt.ylabel("Price (thousands USD)")
plt.xlabel("")
plt.show()

### House Grade

In [ ]:
# distplot of the pricing distribution by grade
fig, ax = plt.subplots(figsize=(16, 9))

sns.boxplot('grade', 'price', data=df, ax=ax)

plt.xlabel('House Grade')
plt.ylabel('House Value (Thousands USD)')
plt.title("Boxplots of House Value by Grade")

# configure y axis
ticks = np.arange(0, df['price'].max() + 100000, 100000)
plt.yticks(ticks, [f"{tick/1000:n}" for tick in ticks])

plt.show()

It appears there is some pattern between House Grade and price. It's extremely obvious by the fact that there is an increasing IQR for each House Grade. We can even see with an anova test, that these populations are completely differently priced.

$\alpha = $ 5%

$H_0: \mu_0 = \mu_1 = \mu_2 = \mu_3 ... = \mu_n$

$H_a: \mu_0 \ne \mu_1 \ne \mu_2 \ne \mu_3 ... \ne \mu_n$

In [ ]:
# Separate our samples by grade into a list
samples = []
for grade in df['grade'].unique():
    mask = df['grade'] == grade
    samples.append(df.loc[mask, 'price'])

In [ ]:
# perform an anova test and output results
results = stats.f_oneway(*samples)
print(f"T-Stat: {results.statistic:.3f}", f"P-Value: {results.pvalue:.8%}", sep='\n')

We can see here that we can reject the null hypothesis that the samples are derived from the same population, our test statistic is so large that our p-value is close to 0. For our stakeholders we will make this simple by
creating a bar plot of the average house value for each house grade.

In [ ]:
# plot bar plot of average house value by grade
fig, ax = plt.subplots(figsize=(16, 9))

sns.barplot('grade', 'price', data=df, ax=ax)
plt.xlabel('House Grade')
plt.ylabel('House Value (Thousands USD)')
plt.title("Average House Value by Grade")

# configure y axis
ticks = np.arange(0, df['price'].max(), 100000)
plt.yticks(ticks, [f"{tick/1000:n}" for tick in ticks])

plt.show()

# Modeling

In [ ]:
# check for any highly correlated variables
fig, ax = plt.subplots(figsize=(16, 9))

mask = np.abs(df.corr()) > .75

sns.heatmap(np.abs(df.corr()), annot=True, fmt=".0%", square=True, ax=ax, mask=~mask)
plt.show()

We will not use sqft_above in our model, as it has a very high correlation with sqft_lot.

## Feature Engineering

> **Only independent/predictor variable(s) is log-transformed**. Divide the coefficient by 100. This tells us that a 1% increase in the independent variable increases (or decreases) the dependent variable by (coefficient/100) units. Example: the coefficient is 0.198. 0.198/100 = 0.00198. For every 1% increase in the independent variable, our dependent variable increases by about 0.002. For x percent increase, multiply the coefficient by log(1.x). Example: For every 10% increase in the independent variable, our dependent variable increases by about 0.198 * log(1.10) = 0.02.  
> \- https://data.library.virginia.edu/interpreting-log-transformations-in-a-linear-model/

In [ ]:
# make two separate lists of our continuous and our categorical features
continuous = ['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15']
categorical = ['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 
               'grade', 'yr_built', 'renovated', 'zipcode', 'basement']

In [ ]:
# display 5 point summary of price
price_sum = df['price'].describe()
display(price_sum.map(lambda val: f"{val:.2f}"))

In [ ]:
# log transform and scale our continuous variables
def normalize(feature):
    return (feature - feature.mean())/feature.std()

cont_log = np.log(df[continuous])
cont_log.columns = [f"{col}_log" for col in cont_log.columns]
cont_log = cont_log.apply(normalize)

# get dummies for categorical columns
cat_cols = pd.get_dummies(df[categorical], drop_first=True)

# combine our preprocessed data
preprocessed = pd.concat([cont_log, cat_cols], axis=1)
X = preprocessed
y = df['price']


## Features Selection

In [ ]:
# import statsmodels and sklearn
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from sklearn.feature_selection import RFE

### Simple Linear Regression

To start our regression we will build a simple linear regression model, which only relies on one feature. To help us decide, we will use a few jointplots.

In [ ]:
# plot jointplot for continuous variables
for col in cont_log.columns:
    sns.jointplot(col, 'price', data=pd.concat([X, y], axis=1), kind='reg')
    
plt.show()

In [ ]:
# split our data
X_train, X_test, y_train, y_test = train_test_split(X[['sqft_living_log']], y, random_state=60)

In [ ]:
# create simple linear model using sqft_living and display summary
simple_model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
display(simple_model.summary())

In [ ]:
# create our model
linreg = LinearRegression()
linreg = linreg.fit(X_train, y_train)

# compute RMSE for our model
mse_train = mean_squared_error(y_train, linreg.predict(X_train))
mse_test = mean_squared_error(y_test, linreg.predict(X_test))

rmse_train, rmse_test = np.sqrt([mse_train, mse_test])
print("RMSE Train:", (rmse_train), "\nRMSE Test:", (rmse_test))

We don't see any major difference between our RMSE scores for our simple model, and our Adj. R-squared is .361.
This model appears to not do the best, so next we will inspect modeling with more features.

In [ ]:
# verify residuals are normally distributed
fig = sm.graphics.qqplot(simple_model.resid, fit=True, line='45')
fig.show()

In [ ]:
fig = plt.Figure(figsize=(16, 9))
sm.graphics.plot_regress_exog(simple_model, 'sqft_living_log', fig=fig)

Important to note, that it appears the residuals of the model are normally distributed. And that there is may be some heteroskedacity. 

## Recursive Feature Selection

In [ ]:
# create model using 10 features
regression = LinearRegression()
selector = RFE(regression, n_features_to_select=10)
selector = selector.fit(X, y)

In [ ]:
# output selected columns
selected_features = X.columns[selector.support_]
print(selected_features)

In [ ]:
# Using 5 kfolds we will cross validate and find the RMSE and R-squared
linreg = LinearRegression()

r2 = cross_validate(linreg, X[selected_features], y, scoring='r2', return_train_score=True)
rmse = cross_validate(linreg, X[selected_features], y, scoring='neg_root_mean_squared_error', return_train_score=True)

print(f"Train Score: r2 = {(r2['train_score'].mean())}, rmse = {(-rmse['train_score'].mean())}")
print(f"Test Score: r2 = {(r2['test_score'].mean())}, rmse = {(-rmse['test_score'].mean())}")

With the selected 10 features, Our R-squared value shows that this model accounts for only 12% of the variation less than the previous 36% in our simple linear model, and our RMSE shows an increase compared to our previous model.

Meaning our previous model did a better job of modeling.

## Step Wise Selection

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            mask = pvalues == worst_pval
            worst_feature = np.array(included)[mask][0]
            try:
                included.remove(worst_feature)
            except:
                pass
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
# split our data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2309814)

In [ ]:
import pickle

with open('stepwise-features.pickle', 'rb') as f:
    selected_features = pickle.load(f)

In [ ]:
# # perform our function over our entire preprocessed dataset
# selected_features = stepwise_selection(X_train, y_train) # very expensive operation

# with open('stepwise-features.pickle', 'wb') as f:
#     pickle.dump(selected_features, f)

In [ ]:
# create our model
model = sm.OLS(y_train, sm.add_constant(X_train[selected_features])).fit()
display(model.summary())

In [ ]:
# Using 10 kfolds we will cross validate and find the RMSE and R-squared
linreg = LinearRegression()

r2 = cross_validate(linreg, X[selected_features], y, scoring='r2', return_train_score=True, cv=10)
rmse = cross_validate(linreg, X[selected_features], y, scoring='neg_root_mean_squared_error', return_train_score=True, cv=10)

print(f"Train Score: r2 = {(r2['train_score'].mean())}, rmse = {(-rmse['train_score'].mean())}")
print(f"Test Score: r2 = {(r2['test_score'].mean())}, rmse = {(-rmse['test_score'].mean())}")

With this model we can see that our R-squared value is much greater than our previous models. 83% of the variation is explained by our model. Our RMSE has also decreased significantly.

In [ ]:
# Let's find out the coefficients for our continuous variables
display((model.params.sort_values(ascending=False)/100).loc[[f"{val}_log" for val in continuous]])

For every 1% increase in sqft_living, we see an increase in house value by \$687.

Meaning, a house with a sqft_living of 1,000, will be worth (206100) + (191516.78) = \$397616.77, with everything else held constant. 

In [ ]:
# top 3 unique independent variables
model.params.sort_values(ascending=False).head(41)

Being in the zipcode 98039, adds $606,524.70 to house value.

Having a house grade of 12, adds $322,168.21 to house value.

Having a house that's near the waterfront, adds $140,481.69 to house value